In [7]:
import sys

!pip install tables
!pip install pycox
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 1.4 MB/s eta 0:00:00


In [3]:
!unzip Archive.zip

Archive:  Archive.zip
   creating: hyperparameters/
  inflating: __MACOSX/._hyperparameters  
  inflating: hyperparameters/METABRIC.xlsx  
  inflating: __MACOSX/hyperparameters/._METABRIC.xlsx  
  inflating: hyperparameters/SEER-CRC-OS.xlsx  
  inflating: __MACOSX/hyperparameters/._SEER-CRC-OS.xlsx  
  inflating: hyperparameters/SUPPORT.xlsx  
  inflating: __MACOSX/hyperparameters/._SUPPORT.xlsx  
  inflating: hyperparameters/SEER-CRC-CRs.xlsx  
  inflating: __MACOSX/hyperparameters/._SEER-CRC-CRs.xlsx  
  inflating: hyperparameters/SEER-Lung-OS.xlsx  
  inflating: __MACOSX/hyperparameters/._SEER-Lung-OS.xlsx  
  inflating: hyperparameters/SEER-Lung-CRs.xlsx  
  inflating: __MACOSX/hyperparameters/._SEER-Lung-CRs.xlsx  
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/metabric_IHC4_clinical_train_test.h5  
  inflating: __MACOSX/data/._metabric_IHC4_clinical_train_test.h5  
   creating: data/metabric/
  inflating: __MACOSX/data/._metabric  
  inflating: data/me

In [8]:
import sys
sys.path.append("../")
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from SADGNet import SADGNetSR
from skopt import BayesSearchCV

In [9]:
# DeepSurv original h5 file has separated folders for x (covariates), t (time-to-event) and e (event)
# so I have to merge them into 1 df

train_data_path = "./data/metabric/train"

y_train_e = pd.read_csv(train_data_path + "/train_e.csv")
y_train_t = pd.read_csv(train_data_path + "/train_t.csv")
x_train = pd.read_csv(train_data_path + "/train_x.csv")

test_data_path = "./data/metabric/test"

y_test_e = pd.read_csv(test_data_path + "/test_e.csv")
y_test_t = pd.read_csv(test_data_path + "/test_t.csv")
x_test = pd.read_csv(test_data_path + "/test_x.csv")


In [10]:
y_train = pd.concat([y_train_t, y_train_e], axis=1)
y_test = pd.concat([y_test_t, y_test_e], axis=1)

In [11]:
# cols_standardize = ['MKI67', 'EGFR', 'PGR', 'ERBB2', 'age at diagnosis']
# cols_leave = ['hormone treatment indicator', 'radiotherapy indicator', 'chemotherapy indicator',
#               'ER-positive indicator']
cols_standardize = ['x0', 'x1', 'x2', 'x3', 'x8']
cols_leave = ['x4', 'x5', 'x6', 'x7']
standardize = [([col], StandardScaler()) for col in cols_standardize]
leave = [(col, None) for col in cols_leave]
x_mapper = DataFrameMapper(standardize + leave)
x_train = x_mapper.fit_transform(x_train).astype('float32')
x_test = x_mapper.transform(x_test).astype('float32')
y_train = y_train.values
y_test = y_test.values

In [12]:
# Tuning hyperparameters
kfold = KFold(n_splits=5, shuffle=True, random_state=2024)
opt = BayesSearchCV(
    SADGNetSR(),
    {
        'learning_rate': (1e-4, 0.01, 'log-uniform'),
        'dropout': [0.5],
        'batch_size': [512],
        'num_layers': [1, 2, 3],
        'hidden_dim': [32, 64, 128],
        'embedding_dim': [32, 64, 128],
        'time_interval': [3, 6, 9],
        'lambda1': [0.5, 1.0, 5.0],
        'lambda2': [1.0, 5.0, 10.0],
        'lambda3': [0.01, 0.1],
        'trans_layer': [1, 2, 3],
        'alpha': [0.5, 1.0, 5.0]
    },
    n_iter=30,
    cv=kfold,
    n_jobs=4
)
opt.fit(x_train, y_train)
print("The best hyperparameters: %s" % opt.best_params_)

/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.01, 0.1] was inferred to Real(low=0.01, high=0.1, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.01, 0.1), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.01, 0.1] was inferred to Real(low=0.01, high=0.1, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, it will be inferred to Categorical(categories=(0.01, 0.1), prior=None). See the documentation of the check_dimension function for the upcoming API.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/skopt/space/space.py:116: UserWarning: Dimension [0.01, 0.1] was inferred to Real(low=0.01, high=0.1, prior='uniform', transform='identity'). In upcoming versions of scikit-optimize, i

AttributeError: `np.Inf` was removed in the NumPy 2.0 release. Use `np.inf` instead.

In [ ]:
# 2025-08-14 21:13:17,559 - SADGNetSR.py[line:197] - INFO: hypeparameters: learning_rate 0.003682096604646654, num_layers 1, hidden_dim 32, dropout 0.5, activation ReLU, batch_size 512, optimizer Adam, embedding_dim 128,time_interval 3,lambda1 5.0, lambda2 1.0, lambda3 0.060309443933746966, trans_layer 2, alpha 5.0


In [ ]:
model = SADGNetSR(**opt.best_params_)
model.fit(x_train, y_train)
c_td = model.score(x_test, y_test)
mae = model.get_mae(x_test, y_test)
print("The C^td of the testset is %.3f" % c_td)
print("The MAE of the testset is %.3f" % mae)

AttributeError: 'BayesSearchCV' object has no attribute 'best_params_'